In [1]:
IMG_SIZE = 512
BATCH_SIZE = 16
EPOCHS = 5
VAL_SPLIT = 0.2

# Data Loading

In [2]:
import tensorflow as tf

dataset_path = "../../data-collection/image-backend/saved_images"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    label_mode='int',
    class_names=['hazard', 'non-hazard'],
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=42,
    validation_split=VAL_SPLIT,
    subset="training",
)

Found 160 files belonging to 2 classes.
Using 128 files for training.


In [3]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    label_mode='int',
    class_names=['hazard', 'non-hazard'],
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=VAL_SPLIT,
    subset="validation",
)

Found 160 files belonging to 2 classes.
Using 32 files for validation.


In [4]:
train_dataset.class_names

['hazard', 'non-hazard']

# Model Fine-tuning

In [5]:
def build_model():
    base_model = tf.keras.applications.EfficientNetB6(
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        weights="imagenet"
    )
    
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    x = tf.keras.applications.efficientnet.preprocess_input(inputs)

    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid", name="pred")(x)

    model = tf.keras.Model(inputs, outputs, name="EfficientNet")

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    
    return model

In [6]:
model = build_model()
model.summary()

Model: "EfficientNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 512, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb6 (Functional)     │ (None, 16, 16, 2304)   │    40,960,143 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2304)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pred (Dense)                    │ (None, 1)              │         2,305 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,962,448 (156.26 MB)

 Trainable params: 2,305 (9.00 KB)

 Non-trainable params: 40,960,143 (156.25 MB)

In [7]:
model.fit(train_dataset, epochs=EPOCHS, verbose=2, validation_data=validation_dataset)

Epoch 1/5
8/8 - 121s - 15s/step - accuracy: 0.7188 - loss: 0.5373 - val_accuracy: 0.7500 - val_loss: 0.5085
Epoch 2/5
8/8 - 106s - 13s/step - accuracy: 0.8828 - loss: 0.2872 - val_accuracy: 0.8125 - val_loss: 0.4124
Epoch 3/5
8/8 - 100s - 12s/step - accuracy: 0.9297 - loss: 0.1740 - val_accuracy: 0.9062 - val_loss: 0.4153
Epoch 4/5
8/8 - 103s - 13s/step - accuracy: 0.9766 - loss: 0.1123 - val_accuracy: 0.8750 - val_loss: 0.3770
Epoch 5/5
8/8 - 106s - 13s/step - accuracy: 0.9766 - loss: 0.0923 - val_accuracy: 0.9062 - val_loss: 0.3845


In [8]:
model.save("../models/EfficientNET.keras")